In [13]:
# 1. 아마존 닷컴 사이트에서 Best Seller 상품의 정보를 추출하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request
import urllib
import time
import pandas as pd    
import os
import math
import pyautogui

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("아마존 닷컴의 분야별 Best Seller 상품 정보 추출하기 ")
print("=" *80)

category = '''1.Amazon Devices & Accessories\t2.Amazon Launchpad\t3.Appliances
4.Apps & Games\t5.Arts, Crafts & Sewing\t6.Audible Books & Originals
7.Automotive\t8.Baby\t9.Beauty & Personal Care
10.Books\t11.Camera & Photo Products\t12.CDs & Vinyl
'''
print(category)

category_input = int(input("1. 위 분야 중에서 자료를 수집할 분야의 번호를 선택하세요: "))
cnt = int(input('2. 해당 분야에서 크롤링 할 건수는 몇 건입니까?(1-100건 사이 입력): '))
page_cnt = math.ceil(cnt/50)

f_dir = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_8\\출력 파일\\"
    
print("\n")

#Step 3.저장될 파일 경로와 이름을 지정합니다

if category_input == 1:
    sec_name = 'Amazon Devices & Accessories'
elif category_input == 2:
    sec_name = 'Amazon Launchpad'
elif category_input == 3:
    sec_name = 'Appliances'
elif category_input == 4:
    sec_name = 'Apps & Games'
elif category_input == 5:
    sec_name = 'Arts, Crafts & Sewing'
elif category_input == 6:
    sec_name = 'Audible Books & Originals'
elif category_input == 7:
    sec_name = 'Automotive'
elif category_input == 8:
    sec_name = 'Baby'
elif category_input == 9:
    sec_name = 'Beauty & Personal Care'
elif category_input == 10:
    sec_name = 'Books'
elif category_input == 11:
    sec_name = 'Camera & Photo Products'
elif category_input == 12:
    sec_name = 'CDs & Vinyl'
    
query_txt='아마존'

n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s1+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s1+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s1+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.xls'

# 제품 이미지 저장용 폴더 생성
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

#Step 4. 웹사이트 접속 후 해당 메뉴로 이동합니다.
s = Service("C:\\chromedriver.exe")
driver = webdriver.Chrome(service=s)

query_url='https://www.amazon.com/bestsellers?Id=NSGoogle'
driver.get(query_url)
time.sleep(4)

html1 = driver.page_source
soup1 = BeautifulSoup(html1, 'html.parser')
go_link = soup1.find("div", "_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz").find_all("a")
go_link = 'https://www.amazon.com' + go_link[category_input - 1]["href"]

driver.get(go_link)
driver.maximize_window()
time.sleep(4)

start = time.time( )

#Step 5. 내용을 수집합니다
ranking2=[]        #제품의 판매순위 저장
name2=[]          #제품이름 저장
price2=[]        #가격 저장
sat_count2 = []     #상품평 수 저장
star2=[]      #평점 저장

img_src2=[]   # 이미지 URL 저장변수
file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수

for x in range(1,page_cnt + 1) :
    if cnt >= 20:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(3)
        
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')

    item_result = soup2.find('div','p13n-gridRow _cDEzb_grid-row_3Cywl').find_all('div', "a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc")

    for li in item_result :
        if cnt < count :
            break
        
        print("%s번째 내용을 추출합니다." % count)

        try :
            photo = li.find('img')['src']
        except AttributeError:
            continue

        file_no += 1
        try:
            urllib.request.urlretrieve(photo, str(file_no)+'.jpg')
        except:
            urllib.request.urlretrieve(urllib.parse.quote(photo.encode('utf8'), '/:'), str(file_no)+'.jpg')
        time.sleep(0.5)

        f = open(ff_name, 'a',encoding='UTF-8')

        ranking = count
        print("1.판매순위:",ranking)
        f.write('1.판매순위:'+ str(ranking) + "\n")
        ranking2.append(ranking)

        try :
            t = li.find('div','_cDEzb_p13n-sc-css-line-clamp-3_g3dy1').get_text().replace("\n","")
        except :
            title = '제품소개가 없습니다'
        else :
            title = t.strip()
            
        print("2.제품이름:", title)
        f.write('2.제품이름:'+ title + "\n")
        name2.append(title)
        
        check_price = li.find_all("div", "a-row")
        
        try :
            try :
                p_price = check_price[1].find('a', 'a-link-normal').get_text()
            except :
                p_price = li.find('a','a-link-normal a-text-normal').get_text().replace("\n","")
        except :
            p_price = '0'      
        else :
            p_price = p_price.strip()
            
        print("3.가격:", p_price)
        f.write('3.가격:'+ p_price + "\n")
        price2.append(p_price)

        try :
            sat_count = li.find('div','a-icon-row').find("span", "a-size-small").get_text().replace("\n","").replace(',', '')
        except  :
            sat_count = '0'
        else :
            sat_count = sat_count.strip()
            
        print("4:상품평 수:", sat_count)
        f.write('4.상품평 수:'+ sat_count + "\n")
        sat_count2.append(sat_count)

        try :
            star = li.find('div','a-icon-row').find("span", "a-icon-alt").get_text().replace("\n","")
        except  :
            star = '0'
        else :
            star = star.strip()
        
        print('5.평점:',star)
        f.write('5.평점:'+ star + "\n")
        f.write("\n")
        star2.append(star)
        
        print("-" *70)

        f.close( )             
        time.sleep(0.5)

        count += 1
        
    x += 1
    
    # 이유는 모르겠으나 chromedriver로 실행시키면 숫자가 사라짐
    try :
        driver.find_element(By.LINK_TEXT, '%s' % x).click()
    except :
        driver.find_element(By.LINK_TEXT, '→').click()
end = time.time( )
        
#step 6. csv , xls 형태로 저장하기              
amazon_best_seller = pd.DataFrame()
amazon_best_seller['판매순위']=ranking2
amazon_best_seller['제품이름']=pd.Series(name2)
amazon_best_seller['가격']=pd.Series(price2)
amazon_best_seller['상품평수']=pd.Series(sat_count2)
amazon_best_seller['평점']=pd.Series(star2)

# csv 형태로 저장하기
amazon_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
amazon_best_seller.to_excel(fx_name ,index=False , engine='openpyxl')
time = end - start

count -= 1
print("\n")
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt, count))
print("2.총 소요시간은 %s 초 입니다 " %round(time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

#Step 7. xls 파일에 제품 이미지 삽입하기
import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api                   #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(2).ColumnWidth = 30 # 셀의 폭 조절
row_cnt = cnt+1 # 1행은 제목
sheet.Rows("2:%s" %row_cnt).RowHeight = 120 # 셀의 높이 조절

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) : # 이미지가 들어갈 column 이름을 지정
    col_name='B'+str(a)
    col_name2.append(col_name)

for b in range(1,cnt+1) : 
    file_name=img_dir+'\\'+str(b)+'.jpg'
    file_name2.append(file_name)
      
for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100) # 파일경로, 링크, 왠만하면 True, 왼쪽, 위쪽, 오ㄹ
    excel.Visible=True # 사진을 넣는 과정을 보여준다.
    excel.ActiveWorkbook.Save() # 합친 결과를 저장한다.

driver.close( )

아마존 닷컴의 분야별 Best Seller 상품 정보 추출하기 
1.Amazon Devices & Accessories	2.Amazon Launchpad	3.Appliances
4.Apps & Games	5.Arts, Crafts & Sewing	6.Audible Books & Originals
7.Automotive	8.Baby	9.Beauty & Personal Care
10.Books	11.Camera & Photo Products	12.CDs & Vinyl

1. 위 분야 중에서 자료를 수집할 분야의 번호를 선택하세요: 1
2. 해당 분야에서 크롤링 할 건수는 몇 건입니까?(1-100건 사이 입력): 10


1번째 내용을 추출합니다.
1.판매순위: 1
2.제품이름: Fire TV Stick with Alexa Voice Remote (includes TV controls), HD streaming device
3.가격: 1 offer from $39.99
4:상품평 수: 231197
5.평점: 4.7 out of 5 stars
----------------------------------------------------------------------
2번째 내용을 추출합니다.
1.판매순위: 2
2.제품이름: Echo Dot (4th Gen) | Sleek design with full sound, Bluetooth, and Alexa | Charcoal
3.가격: $49.99 - $81.95
4:상품평 수: 476085
5.평점: 4.7 out of 5 stars
----------------------------------------------------------------------
3번째 내용을 추출합니다.
1.판매순위: 3
2.제품이름: Echo Dot (4th Gen) | Sleek design with full sound, Bluetooth, and Alexa | Charcoal
3.가격: 1 offer from $49.99
4:상품평 

In [5]:
# 2. 지마켓 사이트에서 Best Seller 상품의 정보를 추출하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request
import urllib
import time
import pandas as pd    
import os
import math
import pyautogui

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("지마켓 닷컴의 분야별 Best Seller 상품 정보 추출하기 ")
print("=" *80)

cnt = int(input('1. 해당 분야에서 크롤링 할 건수는 몇 건입니까?(최대 200건): '))

f_dir = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_8\\출력 파일\\"
    
print("\n")

#Step 3.저장될 파일 경로와 이름을 지정합니다
query_txt='지마켓'
sec_name = '전체'

n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s1+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s1+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s1+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.xls'

# 제품 이미지 저장용 폴더 생성
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

start = time.time( )

#Step 5. 내용을 수집합니다
ranking2=[]        #제품의 판매순위 저장
name2=[]          #제품이름 저장
o_price2=[]        #원래가격 저장
r_price2 = []     #판매가격 저장
rate2=[]      #할인율 저장

img_src2=[]   # 이미지 URL 저장변수
file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수

s = Service("C:\\chromedriver.exe")
driver = webdriver.Chrome(service=s)
query_url='http://corners.gmarket.co.kr/Bestsellers'
driver.get(query_url)
driver.maximize_window()
time.sleep(4)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

item_result = soup.find('div','best-list').find_all('li')

for li in item_result :
    if cnt < count :
        break
        
    print("%s번째 내용을 추출합니다." % count)

    try :
        photo = li.find('img')['src']
        photo = "http:" + photo
    except AttributeError:
        continue

    file_no += 1
    try:
        urllib.request.urlretrieve(photo, str(file_no)+'.jpg')
    except:
        urllib.request.urlretrieve(urllib.parse.quote(photo.encode('utf8'), '/:'), str(file_no)+'.jpg')
    time.sleep(0.5)

    f = open(ff_name, 'a',encoding='UTF-8')

    print("1.판매순위:",count)
    f.write('1.판매순위:'+ str(count) + "\n")
    ranking2.append(count)
    
    try :
        t = li.find('a','itemname').get_text().replace("\n","")
    except :
        title = '제품소개가 없습니다'
    else :
        title = t.strip()
            
    print("2.제품이름:", title)
    f.write('2.제품이름:'+ title + "\n")
    name2.append(title)
        
    try :
        o_price = li.find('div', 'o-price').get_text()
    except :
        o_price = '0원'      
    else :
        o_price = o_price.strip()
    
    if o_price == "":
        o_price = li.find('div', 's-price').get_text().strip()
    
    print("3.원래가격:", o_price)
    f.write('3.원래가격:'+ o_price + "\n")
    o_price2.append(o_price)

    try :
        r_price = li.find('div', 's-price').get_text()
    except  :
        r_price = '0원'
    else :
        r_price = r_price.strip()
            
    print("4:판매가격:", r_price)
    f.write('4.판매가격:'+ r_price + "\n")
    r_price2.append(r_price)

    try :
        rate = li.find('span','sale').get_text().replace("\n","")
    except  :
        rate = '0%'
    else :
        rate = rate.strip()
        
    print("5.할인율:",rate)
    f.write('5.할인율:'+ rate + "\n")
    f.write("\n")
    rate2.append(rate)
        
    print("-" *70)

    f.close( )             
    time.sleep(0.5)

    count += 1
end = time.time( )
        
#step 6. csv , xls 형태로 저장하기              
g_best_seller = pd.DataFrame()
g_best_seller['판매순위']=ranking2
g_best_seller['제품소개']=pd.Series(name2)
g_best_seller['원래가격']=pd.Series(o_price2)
g_best_seller['판매가격']=pd.Series(r_price2)
g_best_seller['할인율']=pd.Series(rate2)

# csv 형태로 저장하기
g_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
g_best_seller.to_excel(fx_name ,index=False , engine='openpyxl')
time = end - start

count -= 1
print("\n")
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt, count))
print("2.총 소요시간은 %s 초 입니다 " %round(time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

#Step 7. xls 파일에 제품 이미지 삽입하기
import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api                   #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(2).ColumnWidth = 30 # 셀의 폭 조절
row_cnt = cnt+1 # 1행은 제목
sheet.Rows("2:%s" %row_cnt).RowHeight = 120 # 셀의 높이 조절

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) : # 이미지가 들어갈 column 이름을 지정
    col_name='B'+str(a)
    col_name2.append(col_name)

for b in range(1,cnt+1) : 
    file_name=img_dir+'\\'+str(b)+'.jpg'
    file_name2.append(file_name)
      
for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100) # 파일경로, 링크, 왠만하면 True, 왼쪽, 위쪽, 오른쪽
    excel.Visible=True # 사진을 넣는 과정을 보여준다.
    excel.ActiveWorkbook.Save() # 합친 결과를 저장한다.

driver.close( )

지마켓 닷컴의 분야별 Best Seller 상품 정보 추출하기 
1. 해당 분야에서 크롤링 할 건수는 몇 건입니까?(최대 200건): 5


1번째 내용을 추출합니다.
1.판매순위: 1
2.제품이름: [마이비]자연품은 얼룩제거제 용기(330ml)+리필(300ml)x3 특가
3.원래가격: 27,000원
4:판매가격: 17,600원
34%
5.할인율: 
----------------------------------------------------------------------
2번째 내용을 추출합니다.
1.판매순위: 2
2.제품이름: [Gfresh]바사삭치킨닭다리 420gx3봉
3.원래가격: 28,500원
4:판매가격: 19,900원
30%
5.할인율: 
----------------------------------------------------------------------
3번째 내용을 추출합니다.
1.판매순위: 3
2.제품이름: (무료반품+12%쿠폰+균일가) 운동복과 일상복까지한 멀티아이템
3.원래가격: 33,000원
4:판매가격: 10,000원
69%
5.할인율: 
----------------------------------------------------------------------
4번째 내용을 추출합니다.
1.판매순위: 4
2.제품이름: 시크루즈/12%/원피스/세트/점프슈트/빅사이즈/바캉스
3.원래가격: 42,600원
4:판매가격: 12,800원
69%
5.할인율: 
----------------------------------------------------------------------
5번째 내용을 추출합니다.
1.판매순위: 5
2.제품이름: [나인웨스트][나인웨스트] 사브리나(Sabrina) 플랫
3.원래가격: 39,000원
4:판매가격: 39,000원
5.할인율: 0%
----------------------------------------------------------------------


1.요청된 총 5 건의 리뷰 중에

In [6]:
# 3. 백그라운드에서 지마켓 사이트에서 Best Seller 상품의 정보를 추출하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request
import urllib
import time
import pandas as pd    
import os
import math
import pyautogui

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("지마켓 닷컴의 분야별 Best Seller 상품 정보 추출하기 ")
print("=" *80)

cnt = int(input('1. 해당 분야에서 크롤링 할 건수는 몇 건입니까?(최대 200건): '))

f_dir = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_8\\출력 파일\\"
    
print("\n")

#Step 3.저장될 파일 경로와 이름을 지정합니다
query_txt='지마켓-백그라운드'
sec_name = '전체'

n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s1+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s1+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s1+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.xls'

# 제품 이미지 저장용 폴더 생성
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

start = time.time( )

#Step 5. 내용을 수집합니다
ranking2=[]        #제품의 판매순위 저장
name2=[]          #제품이름 저장
o_price2=[]        #원래가격 저장
r_price2 = []     #판매가격 저장
rate2=[]      #할인율 저장

img_src2=[]   # 이미지 URL 저장변수
file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수

# 백그라운드 실행
options = webdriver.ChromeOptions()
options.add_argument('headless')
args = ["hide_console", ]
options.add_argument("disable-infobars");
driver = webdriver.Chrome("C:\\chromedriver.exe",options=options,service_args=args)
query_url='http://corners.gmarket.co.kr/Bestsellers'
driver.get(query_url)
driver.maximize_window()
time.sleep(4)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

item_result = soup.find('div','best-list').find_all('li')

for li in item_result :
    if cnt < count :
        break
        
    print("%s번째 내용을 추출합니다." % count)

    try :
        photo = li.find('img')['src']
        photo = "http:" + photo
    except AttributeError:
        continue

    file_no += 1
    try:
        urllib.request.urlretrieve(photo, str(file_no)+'.jpg')
    except:
        urllib.request.urlretrieve(urllib.parse.quote(photo.encode('utf8'), '/:'), str(file_no)+'.jpg')
    time.sleep(0.5)

    f = open(ff_name, 'a',encoding='UTF-8')

    print("1.판매순위:",count)
    f.write('1.판매순위:'+ str(count) + "\n")
    ranking2.append(count)
    
    try :
        t = li.find('a','itemname').get_text().replace("\n","")
    except :
        title = '제품소개가 없습니다'
    else :
        title = t.strip()
            
    print("2.제품이름:", title)
    f.write('2.제품이름:'+ title + "\n")
    name2.append(title)
        
    try :
        o_price = li.find('div', 'o-price').get_text()
    except :
        o_price = '0원'      
    else :
        o_price = o_price.strip()
    
    if o_price == "":
        o_price = li.find('div', 's-price').get_text().strip()
    
    print("3.원래가격:", o_price)
    f.write('3.원래가격:'+ o_price + "\n")
    o_price2.append(o_price)

    try :
        r_price = li.find('div', 's-price').get_text()
    except  :
        r_price = '0원'
    else :
        r_price = r_price.strip()
            
    print("4:판매가격:", r_price)
    f.write('4.판매가격:'+ r_price + "\n")
    r_price2.append(r_price)

    try :
        rate = li.find('span','sale').get_text().replace("\n","")
    except  :
        rate = '0%'
    else :
        rate = rate.strip()
        
    print("5.할인율:",rate)
    f.write('5.할인율:'+ rate + "\n")
    f.write("\n")
    rate2.append(rate)
        
    print("-" *70)

    f.close( )             
    time.sleep(0.5)

    count += 1
end = time.time( )
        
#step 6. csv , xls 형태로 저장하기              
g_best_seller = pd.DataFrame()
g_best_seller['판매순위']=ranking2
g_best_seller['제품소개']=pd.Series(name2)
g_best_seller['원래가격']=pd.Series(o_price2)
g_best_seller['판매가격']=pd.Series(r_price2)
g_best_seller['할인율']=pd.Series(rate2)

# csv 형태로 저장하기
g_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
g_best_seller.to_excel(fx_name ,index=False , engine='openpyxl')
time = end - start

count -= 1
print("\n")
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt, count))
print("2.총 소요시간은 %s 초 입니다 " %round(time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

#Step 7. xls 파일에 제품 이미지 삽입하기
import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api                   #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(2).ColumnWidth = 30 # 셀의 폭 조절
row_cnt = cnt+1 # 1행은 제목
sheet.Rows("2:%s" %row_cnt).RowHeight = 120 # 셀의 높이 조절

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) : # 이미지가 들어갈 column 이름을 지정
    col_name='B'+str(a)
    col_name2.append(col_name)

for b in range(1,cnt+1) : 
    file_name=img_dir+'\\'+str(b)+'.jpg'
    file_name2.append(file_name)
      
for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100) # 파일경로, 링크, 왠만하면 True, 왼쪽, 위쪽, 오른쪽
    excel.Visible=True # 사진을 넣는 과정을 보여준다.
    excel.ActiveWorkbook.Save() # 합친 결과를 저장한다.

driver.close( )

지마켓 닷컴의 분야별 Best Seller 상품 정보 추출하기 
1. 해당 분야에서 크롤링 할 건수는 몇 건입니까?(최대 200건): 5




<ipython-input-6-e871df4b26f4>:66: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\chromedriver.exe",options=options,service_args=args)


1번째 내용을 추출합니다.
1.판매순위: 1
2.제품이름: [마이비]자연품은 얼룩제거제 용기(330ml)+리필(300ml)x3 특가
3.원래가격: 27,000원
4:판매가격: 17,600원
34%
5.할인율: 
----------------------------------------------------------------------
2번째 내용을 추출합니다.
1.판매순위: 2
2.제품이름: [Gfresh]바사삭치킨닭다리 420gx3봉
3.원래가격: 28,500원
4:판매가격: 19,900원
30%
5.할인율: 
----------------------------------------------------------------------
3번째 내용을 추출합니다.
1.판매순위: 3
2.제품이름: (무료반품+12%쿠폰+균일가) 운동복과 일상복까지한 멀티아이템
3.원래가격: 33,000원
4:판매가격: 10,000원
69%
5.할인율: 
----------------------------------------------------------------------
4번째 내용을 추출합니다.
1.판매순위: 4
2.제품이름: 시크루즈/12%/원피스/세트/점프슈트/빅사이즈/바캉스
3.원래가격: 42,600원
4:판매가격: 12,800원
69%
5.할인율: 
----------------------------------------------------------------------
5번째 내용을 추출합니다.
1.판매순위: 5
2.제품이름: [나인웨스트][나인웨스트] 사브리나(Sabrina) 플랫
3.원래가격: 39,000원
4:판매가격: 39,000원
5.할인율: 0%
----------------------------------------------------------------------


1.요청된 총 5 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 5 건입니다
2.총 소요시간은 14.8 초 입니다 
3.파일 저장 완료: txt 파일명 : C:\Users\Win